![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter AI Agent Workshop
# Please go to the end of the document to the cell where all the code is included and run it all at once.

#### **Skills: OpenAI, Groq, Llama, OpenRouter**

## **To Get Started:**
1. [Get your Groq API Key](https://console.groq.com/keys)
2. [Get your OpenRouter API Key](https://openrouter.ai/settings/keys)
3. [Get your OpenAI API Key](https://platform.openai.com/api-keys)


### **Interesting Reads**
- [Sam Altman's Blog Post: The Intelligence Age](https://ia.samaltman.com/)
- [What LLMs cannot do](https://ehudreiter.com/2023/12/11/what-llms-cannot-do/)
- [Chain of Thought Prompting](https://www.promptingguide.ai/techniques/cot)
- [Why ChatGPT can't count the number of r's in the word strawberry](https://prompt.16x.engineer/blog/why-chatgpt-cant-count-rs-in-strawberry)


## During the Workshop
- [Any code shared during the workshop will be posted here](https://docs.google.com/document/d/1hPBJt_4Ihkj6v667fWxVjzwCMS4uBPdYlBLd2IqkxJ0/edit?usp=sharing)

# Install necessary libraries

In [43]:
! pip install openai groq

In [44]:
!pip install requests beautifulsoup4


# Set up Groq, OpenRouter, & OpenAI clients

In [45]:
from openai import OpenAI
from google.colab import userdata
import os
import json
from groq import Groq
import json
from typing import List, Dict, Any, Callable
import ast
import io
import sys
import time
import requests
from bs4 import BeautifulSoup

groq_api_key = userdata.get("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

openrouter_api_key = userdata.get("OPENROUTER_API_KEY")
os.environ['OPENROUTER_API_KEY'] = openrouter_api_key

openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = openai_api_key

groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

openrouter_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

openai_client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

### Define functions to easily query and compare responses from OpenAI, Groq, and OpenRouter

In [4]:
def get_llm_response(client, prompt, openai_model="o1-preview", json_mode=False):

    if client == "openai":

        kwargs = {
            "model": openai_model,
            "messages": [{"role": "user", "content": prompt}]
        }

        if json_mode:
            kwargs["response_format"] = {"type": "json_object"}

        response = openai_client.chat.completions.create(**kwargs)

    elif client == "groq":

        try:
            models = ["llama-3.1-8b-instant", "llama-3.1-70b-versatile", "llama3-70b-8192", "llama3-8b-8192", "gemma2-9b-it"]

            for model in models:

                try:
                    kwargs = {
                        "model": model,
                        "messages": [{"role": "user", "content": prompt}]
                    }
                    if json_mode:
                        kwargs["response_format"] = {"type": "json_object"}

                    response = groq_client.chat.completions.create(**kwargs)

                    break

                except Exception as e:
                    print(f"Error: {e}")
                    continue

        except Exception as e:
            print(f"Error: {e}")

            kwargs = {
                "model": "meta-llama/llama-3.1-8b-instruct:free",
                "messages": [{"role": "user", "content": prompt}]
            }

            if json_mode:
                kwargs["response_format"] = {"type": "json_object"}

            response = openrouter_client.chat.completions.create(**kwargs)

    else:
        raise ValueError(f"Invalid client: {client}")

    return response.choices[0].message.content


def evaluate_responses(prompt, reasoning_prompt=False, openai_model="o1-preview"):
  # Here, I could append the response from the webscraper ##########################################################################

    if reasoning_prompt:
        prompt = f"{prompt}\n\n{reasoning_prompt}."

    openai_response = get_llm_response("openai", prompt, openai_model)
    groq_response = get_llm_response("groq", prompt)

    print(f"OpenAI Response: {openai_response}")
    print(f"\n\nGroq Response: {groq_response}")

In [6]:
prompt = "Which is greater, 9.11 or 9.9?"
# Allows the model to think critically
reasoning_prompt = "Understand the problem and devise a plan to solve it. Then solve the problem step by step"
evaluate_responses(prompt, reasoning_prompt, openai_model="gpt-3.5-turbo")

OpenAI Response: To determine which is greater between 9.11 and 9.9, we need to compare the numbers digit by digit starting from the left.

9.11
9.90

Since both numbers have the same digit at the first decimal place, we move on to the second decimal place.

9.11
9.90

Comparing the numbers at the second decimal place, we see that 9.9 is greater than 9.11.

Therefore, 9.9 is greater than 9.11.


Groq Response: To determine which number is greater between 9.11 and 9.9, we can use a simple comparison. Here's the step-by-step plan:

1. Compare the whole numbers (9 in both cases).
2. Compare the decimal parts (0.11 and 0.9).

**Step-by-Step Solution**

1. Compare the whole numbers:
Both numbers have the same whole number part, which is 9.
2. Compare the decimal parts:
- The decimal part of 9.11 is 0.11.
- The decimal part of 9.9 is 0.9.
Since 0.9 is greater than 0.11, 
Therefore, 9.9 is greater than 9.11.


# Agent Architecture

[![](https://mermaid.ink/img/pako:eNqFUslugzAQ_ZWRz8kPILUVCZClVdQmqdTK5ODCFFDAjry0iSD_XmOTJodK5WS_Zd7M4JZkIkcSkEKyQwnbKOVgvzf6qlDCi0F52sF4fA-hJ0IaGi24aIRREBbItacn9LlmnKPced2kR7s1aiO5AmU-NFN71cG0fRLiALqUwhQlbAbi7F1TVyuia2RKXItFDo5pmOnqBo5dhgcDmKFlmEaY2oE6SOgvwHgO8REzM5B_2n1kxYsOZtSrxSUocfkzf5m5y5zGX6w27Cqau3IDOpTU8gR3kLBa2Y4W7QqP-jLywzDywtnesd_NLbISHSxpUnFWQ8jVt_0b8VFLll0Tl66TR-q3DLfaf3vaSoMukYxIg7JhVW4fQdvbUqJLbDAlgT3mTO5TkvKz1TG7ks2JZyTQ1j0i5pDb9UYVs2-nIcFnP-YFjfPKNjqA5x9CM8YW?type=png)](https://mermaid.live/edit#pako:eNqFUslugzAQ_ZWRz8kPILUVCZClVdQmqdTK5ODCFFDAjry0iSD_XmOTJodK5WS_Zd7M4JZkIkcSkEKyQwnbKOVgvzf6qlDCi0F52sF4fA-hJ0IaGi24aIRREBbItacn9LlmnKPced2kR7s1aiO5AmU-NFN71cG0fRLiALqUwhQlbAbi7F1TVyuia2RKXItFDo5pmOnqBo5dhgcDmKFlmEaY2oE6SOgvwHgO8REzM5B_2n1kxYsOZtSrxSUocfkzf5m5y5zGX6w27Cqau3IDOpTU8gR3kLBa2Y4W7QqP-jLywzDywtnesd_NLbISHSxpUnFWQ8jVt_0b8VFLll0Tl66TR-q3DLfaf3vaSoMukYxIg7JhVW4fQdvbUqJLbDAlgT3mTO5TkvKz1TG7ks2JZyTQ1j0i5pDb9UYVs2-nIcFnP-YFjfPKNjqA5x9CM8YW)


![agent_architecture_v2](https://github.com/user-attachments/assets/a65b6db9-bef1-4579-aed3-01444ce40544)

### To create our AI Agent, we will define the following functions:

1. **Planner:** This function takes a user's query and breaks it down into smaller, manageable subtasks. It returns these subtasks as a list, where each one is either a reasoning task or a code generation task.

2. **Reasoner:** This function provides reasoning on how to complete a specific subtask, considering both the overall query and the results of any previous subtasks. It returns a short explanation on how to proceed with the current subtask.

3. **Actioner:** Based on the reasoning provided for a subtask, this function decides whether the next step requires generating code or more reasoning. It then returns the chosen action and any necessary details to perform it.

4. **Evaluator:** This function checks if the result of the current subtask is reasonable and aligns with the overall goal. It returns an evaluation of the result and indicates whether the subtask needs to be retried.

5. **generate_and_execute_code:** This function generates and executes Python code based on a given prompt and memory of previous steps. It returns both the generated code and its execution result.

6. **executor:** Depending on the action decided by the "actioner," this function either generates and executes code or returns reasoning. It handles the execution of tasks based on the action type.

7. **final_answer_extractor:** After all subtasks are completed, this function gathers the results from previous steps to extract and provide the final answer to the user's query.

8. **autonomous_agent:** This is the main function that coordinates the process of answering the user's query. It manages the entire sequence of planning, reasoning, action, evaluation, and final answer extraction to produce a complete response.

**Web Scraper Function**

In [17]:
# Initialize the web scraper function. I will store the response so that the planner can reference it.
def web_scraper(url):
    response = requests.get(url)

    if response.status_code == 200:
      soup = BeautifulSoup(response.content, "html.parser")
      title = soup.title.string if soup.title else "No title found"
      meta_description = soup.find("meta", attrs={"name": "description"})
      description = meta_description["content"] if meta_description else "No description found"

      return {"title": title, "description": description}

    else:

      return {"error": f"Failed to fetch the website. Status code: {response.status_code}"}

    scraped_data = web_scraper(url)
    if "error" in scraped_data:
        return scraped_data["error"]

    response = f"The title of the page is '{scraped_data['title']}' and the description is '{scraped_data['description']}' "
    scraped_title = scraped_data['title']
    scraped_data = scraped_data['description']
    return response, scraped_title, scraped_data

**Planner Function**

In [27]:
def planner(user_query: str) -> List[str]:
   prompt = f"""Given the user's query: '{user_query}', break down the query into as few subtasks as possible in order to anser the question.
   Each subtask is either a calculation or reasoning step. Never duplicate a task.

   Each subtask can either:
       - generate_code: This action involves generating Python code and executing it in order to make a calculation or verification.
       - reasoning: This action involves providing reasoning for what to do to complete the subtask.

   Each subtask should begin with either "reasoning" or "generate_code".

   Keep in mind the overall goal of answering the user's query throughout the planning process.

   Return the result as a JSON list of strings, where each string is a subtask.

   Here is an example JSON response:

   {{
       "subtasks": ["Subtask 1", "Subtask 2", "Subtask 3"]
   }}
   """
    # I did not create another task for web scraping since the scraped_data also falls under the reasoning category, such as finding specific info that answers the user's prompt.
   reasoning_prompt = """
   First, analyse and devise a step by step solution.
   Given the web scraper's response: Title: '{scraped_title}' and  Description: '{scraped_description}',  use relevant information to complete a subtask.
   Do not include any other sources of information.
   Provide results relevant to the user's query.
   """
   response = json.loads(get_llm_response("groq", prompt, reasoning_prompt, json_mode=True)) # These are the parameters input to the LLM.
   print(response)
   return response["subtasks"]

**Reasoner Function**

In [19]:
def reasoner(user_query: str, subtasks: List[str], current_subtask: str, memory: List[Dict[str, Any]]) -> str:
   prompt = f"""Given the user's query (long-term goal): '{user_query}',

   here are all the subtasks to complete in order to answer the user's query:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   Here is the short-term memory (result of previous subtasks):
   <memory>
       {json.dumps(memory)}
   </memory>

   The current subtask to complete is:
   <current_subtask>
       {current_subtask}
   </current_subtask>

   - Provide concise reasoning on how to execute the current subtask, considering previous results where relevant.
   - Prioritize explicit details over assumed patterns
   - Avoid unnecessary complications in problem-solving.
   - Keep the solution straightforward, without repeating subtasks.

   Return the result as a JSON object with 'reasoning' as a key.

   Example JSON response:
   {{
       "reasoning": "2 sentences max on how to complete the current subtask."
   }}
   """

   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response["reasoning"]

**Actioner Function**

In [20]:
def actioner(user_query: str, subtasks: List[str], current_subtask: str, reasoning: str, memory: List[Dict[str, Any]]) -> Dict[str, Any]:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   The subtasks are:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   The current subtask is:
   <current_subtask>
       {current_subtask}
   </current_subtask>

   The reasoning for this subtask is:
   <reasoning>
       {reasoning}
   </reasoning>

   Here is the short-term memory (result of previous subtasks):
   <memory>
       {json.dumps(memory)}
   </memory>

   Determine the most appropriate action to take:
       - If the task requires a calculation or verification through code, use the 'generate_code' action.
       - If the task requires reasoning without code or calculation, use the 'reasoning' action.

   Consider the overall goal and previous results when determining the action.

   Return the result as a JSON object with 'action' and 'parameters' keys.  The 'parameters' key should always be a dictionary with 'prompt' as a key.

   Example JSON responses:

   {{
       "action": "generate_code",
       "parameters": {{"prompt": "Write a function to calculate the area of a circle."}}
   }}

   {{
       "action": "reasoning",
       "parameters": {{"prompt": "Explain how to complete the subtask."}}
   }}
   """

   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response

**Evaluator Function**

In [21]:
def evaluator(user_query: str, subtasks: List[str], current_subtask: str, action_info: Dict[str, Any], execution_result: Dict[str, Any], memory: List[Dict[str, Any]]) -> Dict[str, Any]:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   The subtasks to complete to answer the user's query are:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   The current subtask to complete is:
   <current_subtask>
       {current_subtask}
   </current_subtask>

   The result of the current subtask is:
   <result>
       {action_info}
   </result>

   The execution result of the current subtask is:
   <execution_result>
       {execution_result}
   </execution_result>

   Here is the short-term memory (result of previous subtasks):
   <memory>
       {json.dumps(memory)}
   </memory>


   Evaluate if the result is a reasonable answer for the current subtask, and makes sense in the context of the user's query.

   Return a JSON object with 'evaluation' (string) and 'retry' (boolean) keys.

   Example JSON response:
   {{
       "evaluation": "The result is a reasonable answer for the current subtask.",
       "retry": false
   }}
   """


   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response


**Extractor Function**

In [22]:
def final_answer_extractor(user_query: str, subtasks: List[str], memory: List[Dict[str, Any]]) -> str:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   The subtasks completed to answer the user's query are:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   The memory of the thought process (short-term memory) is:
   <memory>
       {json.dumps(memory)}
   </memory>

   Extract the final answer that addresses the user's query, from the memory.
   Provide only the essential information without unnecessary explanations.

   Return a JSON object with 'finalAnswer' as a key.

   Here is an example JSON response:
   {{
       "finalAnswer": "The final answer to the user's query, addressing all aspects of the question, based on the memory provided",
   }}
   """

   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response["finalAnswer"]


**Code Generation Function**

In [23]:
def generate_and_execute_code(prompt: str, user_query: str, memory: List[Dict[str, Any]]) -> Dict[str, Any]:
   code_generation_prompt = f"""

   Only for a generate_code subtask:
   - Generate Python code to implement the following task: '{prompt}'

   - The overall goal is to answer the user's query: '{user_query}'

   - Keep in mind the results of the previous subtasks, and use them to complete the current subtask.
    <memory>
        {json.dumps(memory)}
    </memory>



  -  Here are the guidelines for generating the code:
        - Provide only the Python code, without explanations or markdown.
        - The code must print or return a value.
        - Don't include any backticks, code blocks or markdown like ```python or ``` in your response, just give me the code.
        - Do not ever use the input() function in your code, use defined values instead.
        - Do not ever import NLP libraries in your code, such as nltk, spacy, or any other NLP library.
        - Generate straight-forward executable code, do not ever use functions.
        - Don't ever provide the execution result in your response.
        - Import all required libraries within the code itself.
        - The code should be self-contained and ready to execute.
        - Prioritize explicit details over assumed patterns.
        - Keep the solution simple and efficient.
    """

   generated_code = get_llm_response("groq", code_generation_prompt)


   print(f"\n\nGenerated Code: start|{generated_code}|END\n\n")

   old_stdout = sys.stdout
   sys.stdout = buffer = io.StringIO()

   exec(generated_code)

   sys.stdout = old_stdout
   output = buffer.getvalue()

   print(f"\n\n***** Execution Result: |start|{output.strip()}|end| *****\n\n")

   return {
       "generated_code": generated_code,
       "execution_result": output.strip()
   }


**Executor Function**

In [24]:
def executor(action: str, parameters: Dict[str, Any], user_query: str, memory: List[Dict[str, Any]]) -> Any:
   if action == "generate_code":
       print(f"Generating code for: {parameters['prompt']}")
       return generate_and_execute_code(parameters["prompt"], user_query, memory)
   elif action == "reasoning":
       return parameters["prompt"]
   else:
       return f"Action '{action}' not implemented"

**Autonomous Agent Function**

In [25]:
def autonomous_agent(user_query: str) -> List[Dict[str, Any]]:
   memory = []
   subtasks = planner(user_query)

   print("User Query:", user_query)
   print(f"Subtasks: {subtasks}")

   for subtask in subtasks:
       max_retries = 1
       for attempt in range(max_retries):

           reasoning = reasoner(user_query, subtasks, subtask, memory)
           action_info = actioner(user_query, subtasks, subtask, reasoning, memory)


           print(f"\n\n ****** Action Info: {action_info} ****** \n\n")

           execution_result = executor(action_info["action"], action_info["parameters"], user_query, memory)

           print(f"\n\n ****** Execution Result: {execution_result} ****** \n\n")
           evaluation = evaluator(user_query, subtasks, subtask, action_info, execution_result, memory)

           step = {
               "subtask": subtask,
               "reasoning": reasoning,
               "action": action_info,
               "evaluation": evaluation
           }
           memory.append(step)

           print(f"\n\nSTEP: {step}\n\n")

           if not evaluation["retry"]:
               break

           if attempt == max_retries - 1:
               print(f"Max retries reached for subtask: {subtask}")

   final_answer = final_answer_extractor(user_query, subtasks, memory)
   return final_answer

# **Running all the code at once**

In [59]:
import requests
from bs4 import BeautifulSoup
import json
import sys
import io
from typing import List, Dict, Any

# Web Scraper function
def web_scraper(url: str) -> dict:
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.title.string if soup.title else "No title found"
        meta_description = soup.find("meta", attrs={"name": "description"})
        description = meta_description["content"] if meta_description else "No description found"
        return {"title": title, "description": description}
    else:
        return {"error": f"Failed to fetch the website. Status code: {response.status_code}"}

# Planner function
def planner(user_query: str, scraped_title: str, scraped_description: str) -> List[str]:
    prompt = f"""Given the user's query: '{user_query}', break down the query into as few subtasks as possible in order to answer the question.
    Each subtask is either a calculation or reasoning step. Never duplicate a task.

    Each subtask can either:
        - generate_code: This action involves generating Python code and executing it in order to make a calculation or verification.
        - reasoning: This action involves providing reasoning for what to do to complete the subtask.

    Each subtask should begin with either "reasoning" or "generate_code".

    Keep in mind the overall goal of answering the user's query throughout the planning process.

    Return the result as a JSON list of strings, where each string is a subtask.
    """

    reasoning_prompt = f"""
    First, analyze and devise a step-by-step solution.
    Given the web scraper's response: Title: '{scraped_title}' and Description: '{scraped_description}', use relevant information to complete a subtask.
    Provide results relevant to the user's query.
    """

    # Simulated response from LLM
    response = json.loads(get_llm_response("groq", prompt, reasoning_prompt, json_mode=True))
    return response["subtasks"]

# Reasoner Function
def reasoner(user_query: str, subtasks: List[str], current_subtask: str, memory: List[Dict[str, Any]]) -> str:
    prompt = f"""Given the user's query (long-term goal): '{user_query}',
    here are all the subtasks to complete in order to answer the user's query:
    <subtasks>
        {json.dumps(subtasks)}
    </subtasks>

    Here is the short-term memory (result of previous subtasks):
    <memory>
        {json.dumps(memory)}
    </memory>

    The current subtask to complete is:
    <current_subtask>
        {current_subtask}
    </current_subtask>

    - Provide concise reasoning on how to execute the current subtask, considering previous results where relevant.
    - Prioritize explicit details over assumed patterns
    - Avoid unnecessary complications in problem-solving.
    - Keep the solution straightforward, without repeating subtasks.

    Return the result as a JSON object with 'reasoning' as a key.
    """
    response = json.loads(get_llm_response("groq", prompt, json_mode=True))
    return response["reasoning"]

# Actioner Function
# Actioner Function
def actioner(user_query: str, subtasks: List[str], current_subtask: str, reasoning: str, memory: List[Dict[str, Any]]) -> Dict[str, Any]:
    prompt = f"""Given the user's query (long-term goal): '{user_query}'

    The subtasks are:
    <subtasks>
        {json.dumps(subtasks)}
    </subtasks>

    The current subtask is:
    <current_subtask>
        {current_subtask}
    </current_subtask>

    The reasoning for this subtask is:
    <reasoning>
        {reasoning}
    </reasoning>

    Here is the short-term memory (result of previous subtasks):
    <memory>
        {json.dumps(memory)}
    </memory>

    Determine the most appropriate action to take for each subtask.:
        - If the task requires a calculation or verification through code, use the 'generate_code' action.
        - If the task requires a response without code or calculation, use the 'reasoning' action.

    Consider the overall goal and previous results when determining the action.

    Return the result as a JSON object with 'action' and 'parameters' keys. The 'parameters' key should always be a dictionary with 'prompt' as a key.

    Example JSON responses:
    {{
        "action": "generate_code",
        "parameters": {{"prompt": "Write a function to calculate the area of a circle."}}
    }}

    {{
        "action": "reasoning",
        "parameters": {{"prompt": "Explain how to complete the subtask."}}
    }}
    """

    response = json.loads(get_llm_response("groq", prompt, json_mode=True))

    # Check if 'action' and 'parameters' keys exist in response
    if 'action' not in response:
        print(f"Warning: 'action' key not found in response from LLM: {response}")
        response['action'] = 'reasoning'  # Default action if not found

    if 'parameters' not in response:
        print(f"Warning: 'parameters' key not found in response from LLM: {response}")
        response['parameters'] = {'prompt': 'Provide reasoning for the current subtask.'}  # Default parameters

    return response


# Evaluator Function
def evaluator(user_query: str, subtasks: List[str], current_subtask: str, action_info: Dict[str, Any], execution_result: Dict[str, Any], memory: List[Dict[str, Any]]) -> Dict[str, Any]:
    prompt = f"""Given the user's query (long-term goal): '{user_query}'
    The subtasks to complete to answer the user's query are:
    <subtasks>
        {json.dumps(subtasks)}
    </subtasks>

    The current subtask to complete is:
    <current_subtask>
        {current_subtask}
    </current_subtask>

    The result of the current subtask is:
    <result>
        {action_info}
    </result>

    The execution result of the current subtask is:
    <execution_result>
        {execution_result}
    </execution_result>

    Here is the short-term memory (result of previous subtasks):
    <memory>
        {json.dumps(memory)}
    </memory>

    Evaluate if the result is a reasonable answer for the current subtask, and makes sense in the context of the user's query.

    Return a JSON object with 'evaluation' (string) and 'retry' (boolean) keys.
    """
    response = json.loads(get_llm_response("groq", prompt, json_mode=True))
    return response

# Extracting the final answer
def final_answer_extractor(user_query: str, subtasks: List[str], memory: List[Dict[str, Any]]) -> str:
    prompt = f"""Given the user's query (long-term goal): '{user_query}'
    The subtasks completed to answer the user's query are:
    <subtasks>
        {json.dumps(subtasks)}
    </subtasks>

    The memory of the thought process (short-term memory) is:
    <memory>
        {json.dumps(memory)}
    </memory>

    Extract the final answer that addresses the user's query, from the memory.
    Provide only the essential information without unnecessary explanations.

    Return a JSON object with 'finalAnswer' as a key.
    """
    response = json.loads(get_llm_response("groq", prompt, json_mode=True))
    return response["finalAnswer"]

# Code Generator and Executor Function
def generate_and_execute_code(prompt: str, user_query: str, memory: List[Dict[str, Any]]) -> Dict[str, Any]:
    code_generation_prompt = f"""
    Only for a generate_code subtask:
    - Generate Python code to implement the following task: '{prompt}'
    - The overall goal is to answer the user's query: '{user_query}'
    - Keep in mind the results of the previous subtasks, and use them to complete the current subtask.
    <memory>
        {json.dumps(memory)}
    </memory>

    Here are the guidelines for generating the code:
        - Provide only the Python code, without explanations or markdown.
        - The code must print or return a value.
        - Don't include any backticks, code blocks or markdown like ```python or ``` in your response, just give me the code.
        - Do not ever use the input() function in your code, use defined values instead.
        - Do not ever import NLP libraries in your code, such as nltk, spacy, or any other NLP library.
        - Generate straight-forward executable code, do not ever use functions.
        - Don't ever provide the execution result in your response.
        - Import all required libraries within the code itself.
        - The code should be self-contained and ready to execute.
        - Prioritize explicit details over assumed patterns.
        - Keep the solution simple and efficient.
    """

    generated_code = get_llm_response("groq", code_generation_prompt)

    print(f"\n\nGenerated Code: start|{generated_code}|END\n\n")

    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()

    exec(generated_code)

    sys.stdout = old_stdout
    output = buffer.getvalue()

    print(f"\n\n***** Execution Result: |start|{output.strip()}|end| *****\n\n")

    return {
        "generated_code": generated_code,
        "execution_result": output.strip()
    }

# Executor function
def executor(action: str, parameters: Dict[str, Any], user_query: str, memory: List[Dict[str, Any]]) -> Any:
    if action == "generate_code":
        print(f"Generating code for: {parameters.get('prompt', 'No prompt provided')}")
        return generate_and_execute_code(parameters.get("prompt", ''), user_query, memory)
    elif action == "reasoning":
        return parameters.get("prompt", 'No reasoning prompt provided')
    else:
        return f"Action '{action}' not implemented"


# Autonomous Agent
def autonomous_agent(user_query: str) -> List[Dict[str, Any]]:
    memory = []

    # Web scraping
    url = "https://blog.samaltman.com/what-i-wish-someone-had-told-me"
    scraped_data = web_scraper(url)

    if "error" in scraped_data:
        print(scraped_data["error"])  # Handle the error
        return []

    scraped_title = scraped_data['title']
    scraped_description = scraped_data['description']

    # Planning
    subtasks = planner(user_query, scraped_title, scraped_description)

    # Iterating through each subtask
    for subtask in subtasks:
        current_subtask = subtask
        reasoning = reasoner(user_query, subtasks, current_subtask, memory)
        action_info = actioner(user_query, subtasks, current_subtask, reasoning, memory)

        execution_result = executor(action_info['action'], action_info['parameters'], user_query, memory)

        # Evaluating
        evaluation_result = evaluator(user_query, subtasks, current_subtask, action_info, execution_result, memory)

        if evaluation_result['retry']:
            print(f"Retrying subtask: {current_subtask}")
            # Logic for retrying if necessary (not implemented in this example)

        # Append to memory
        memory.append({
            "subtask": current_subtask,
            "reasoning": reasoning,
            "action_info": action_info,
            "execution_result": execution_result,
            "evaluation_result": evaluation_result
        })

    # Extracting final answer
    final_answer = final_answer_extractor(user_query, subtasks, memory)
    return final_answer

# Testing the agent
user_query = "What does Sam Altman say in the blog What I Wish Someone Had Told Me?"
final_result = autonomous_agent(user_query)
final_result


'Sam Altman suggests qualities of an effective mentor that one wishes someone had told them when they were starting their journey, such as seeking mentors from a diverse set, avoiding common pitfalls, and maintaining a growth mindset.'

# OpenAI o1-preview model getting trivial questions wrong

- [Link to Reddit post](https://www.reddit.com/r/ChatGPT/comments/1ff9w7y/new_o1_still_fails_miserably_at_trivial_questions/)
- Links to ChatGPT threads: [(1)](https://chatgpt.com/share/66f21757-db2c-8012-8b0a-11224aed0c29), [(2)](https://chatgpt.com/share/66e3c1e5-ae00-8007-8820-fee9eb61eae5)
- [Improving reasoning in LLMs through thoughtful prompting](https://www.reddit.com/r/singularity/comments/1fdhs2m/did_i_just_fix_the_data_overfitting_problem_in/?share_id=6DsDLJUu1qEx_bsqFDC8a&utm_content=2&utm_medium=ios_app&utm_name=ioscss&utm_source=share&utm_term=1)


**Testing the autonomous agent**